In [1]:
import sys
import os

repo_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.insert(0, repo_root)

In [2]:
working_dir  = '/home/yunan/foreground_test/combined/'

In [3]:
import numpy as np
import pandas as pd

from tqdm import tqdm
import matplotlib.pyplot as plt

import healpy as hp
import pysm3
import pysm3.units as u

In [4]:
prop_cycle = plt.rcParams['axes.prop_cycle']
colors = prop_cycle.by_key()['color']
del prop_cycle  # clean up namespace

In [5]:
# Swallow downgrade errors.  TODO: Why are there downgrade errors?
import logging

class LoggingContextManager:
    def __init__(self, filename, level=logging.WARNING, exit_msg=None):
        self.filename = filename
        self.level = level
        self.exit_msg = exit_msg
        self.first_issue_notified = False
        self.issue_occurred = False
        self.logger = logging.getLogger()
        self.file_handler = logging.FileHandler(filename)
        self.file_handler.setLevel(level)
        self.file_handler.setFormatter(logging.Formatter('%(asctime)s - %(levelname)s - %(message)s'))
        self.original_handlers = None

    def __enter__(self):
        self.original_handlers = self.logger.handlers[:]
        # Set the logger to the lowest possible level during the context to ensure all messages are processed
        self.logger.setLevel(logging.DEBUG)
        self.logger.handlers = []  # Remove existing handlers to avoid duplicate logs
        self.logger.addHandler(self.file_handler)
        self.logger.addFilter(self.process_notification)  # Add custom filter
        return self

    def __exit__(self, exc_type, exc_value, traceback):
        self.logger.removeHandler(self.file_handler)
        self.logger.handlers = self.original_handlers  # Restore original handlers
        self.file_handler.close()
        if self.issue_occurred:
            print(self.exit_msg or "End of processing: Issues were logged during the session.")

    def process_notification(self, record):
        """Custom filter to process notifications for the first issue."""
        if record.levelno >= self.level:
            if not self.first_issue_notified:
                print(f"First issue encountered; check {self.filename} for more information.")
                self.first_issue_notified = True
            self.issue_occurred = True
        return True  # Always return True to ensure all messages are logged

# Setup basic configuration for logging
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s')

In [6]:
nside_sky = 2048
nside_out = 512

output_dir = os.path.join(working_dir, f"sky{nside_sky}_out{nside_out}")
os.makedirs(output_dir, exist_ok=True)

In [7]:
target_freqs = [30, 44, 70, 100, 143, 217, 353, 545, 857]
target_freqs = [f * u.GHz for f in target_freqs]

In [8]:
components = ['d9', 's4', 'f1', 'a1', 'co1', 'cib1', 'ksz1', 'tsz1', 'rg1']
#["d9", "s4", "f1", "a1", "co1", "cib1", "ksz1", "tsz1", "rg1", "d1", "s1", "c1"]

In [9]:
#lmax = int(3 * nside_sky - 1)
#lmax_out = int(3 * nside_out - 1)
#lmax might be too high
beam_fwhm = { '30.0': 75, '44.0': 65, '70.0': 55, '100.0': 43, '143.0': 32.4, '217.0': 22.3, '353.0': 22.0, '545.0': 21.5, '857.0': 20.6 }
beam_fwhm = { k: v * u.arcmin for k, v in beam_fwhm.items() }

In [16]:
# # Produce maps, save them to disk, for limited RAM usage
# # map_dict = {}
# n_sims = 1
# pbar = tqdm(total=n_sims * len(components) * len(target_freqs), desc="Processing components and frequencies")

# # PySM3 throws a warning for each of the calls to apply_smoothing_and_coord_transform(). I'm ok with the lack of convergence.
# with LoggingContextManager("pysm3_warnings.log", exit_msg="End of processing: Warnings were logged during the session.") as log:
#     for sim_num in range(n_sims):
#         np.random.seed(sim_num)
#         for comp in components:
#             sky = pysm3.Sky(nside=nside_sky, preset_strings=[comp])
#             for freq in target_freqs:
#                 sky_observed = sky.get_emission(freq)
#                 if nside_sky != nside_out:
#                     # Downgrade the map to the output nside; PySM3 has this as a catch-all function, because it operates in alm space internally
#                     sky_map = pysm3.apply_smoothing_and_coord_transform(sky_observed[0],
#                                                                         fwhm=beam_fwhm[str(freq.value)],
#                                                                         lmax=lmax,
#                                                                         output_nside=nside_out)
#                     sky_map = sky_map.to(u.K_CMB, equivalencies=u.cmb_equivalencies(freq))
#                 np.save(os.path.join(output_dir, f"sim{sim_num}_{comp}_{freq.value}GHz.npy"), np.array(sky_map.data))
#                 hp.write_map(os.path.join(output_dir, f"sim{sim_num}_{comp}_{freq}.fits"), np.array(sky_map.data), overwrite=True)
#                 pbar.update(1)
# del sky, sky_observed, freq, comp, pbar  # Clean up namespace

Processing components and frequencies:   0%|          | 0/9 [00:00<?, ?it/s]

Processing components and frequencies: 100%|██████████| 9/9 [38:14<00:00, 254.92s/it]


In [10]:
n_sims = 10  # Number of simulations
combined_maps = {}  # Dictionary to store combined maps
pbar = tqdm(total=n_sims * len(target_freqs), desc="Processing combined maps for frequencies")

# PySM3 throws warnings for smoothing; log them.
with LoggingContextManager("pysm3_warnings.log", exit_msg="End of processing: Warnings were logged during the session.") as log:
    for sim_num in range(n_sims):
        np.random.seed(sim_num)
        for freq in target_freqs:
            combined_map = np.zeros(hp.nside2npix(nside_out))  # Initialize combined map with zeros
            
            # Process each component and add its contribution to the combined map
            # for comp in components:
            sky = pysm3.Sky(nside=nside_sky, preset_strings= components)
            sky_observed = sky.get_emission(freq)
             
            #Lmax could be left for healpy to handle
            if nside_sky != nside_out:
                sky_map = pysm3.apply_smoothing_and_coord_transform(
                    sky_observed[0],
                    fwhm=beam_fwhm[str(freq.value)],
                    #lmax=lmax,
                    output_nside=nside_out
                )
                sky_map = sky_map.to(u.K_CMB, equivalencies=u.cmb_equivalencies(freq))
            else:
                sky_map = sky_observed[0]
             
            # Add this component's contribution to the combined map
            # combined_map += sky_map.value

            # Save combined map
            np.save(os.path.join(output_dir, f"sim{sim_num}_combined_{freq.value}GHz.npy"), sky_map.value)
            hp.write_map(os.path.join(output_dir, f"sim{sim_num}_combined_{freq.value}GHz.fits"), sky_map, overwrite=True)
            pbar.update(1)

# Clean up namespace
del sky, sky_observed, freq, components, pbar, combined_map

Processing combined maps for frequencies:   0%|          | 0/90 [00:00<?, ?it/s]

Processing combined maps for frequencies: 100%|██████████| 90/90 [4:06:34<00:00, 164.38s/it]  


In [19]:
import os
import glob

# Define the root directory and simulation folder template
root_dir = "/home/yunan/pyilc/output/foreground/co1/"
sim_folder_template = "sim*"

# Define the filename templates
fn_template1 = "CN__needletcoeff_covmap_freq*_freq*_scale*.fits"
fn_template2 = "CN__needletcoeffmap_freq*_scale*.fits"
fn_template3 = "CN_needletILCmap_scale*_component_CMB_includechannels*_noise_res.fits"

# List of filename templates to delete
templates_to_delete = [fn_template1, fn_template2, fn_template3]

def delete_files(root_dir, sim_folder_template, templates_to_delete):
    # Find all simulation folders in the root directory
    sim_folders = [os.path.join(root_dir, d) for d in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, d)) and d.startswith("sim")]

    for sim_folder in sim_folders:
        print(f"Processing simulation folder: {sim_folder}")

        for template in templates_to_delete:
            # Generate the pattern for glob
            pattern = os.path.join(sim_folder, template)
            
            # Find files matching the pattern
            files_to_delete = glob.glob(pattern)

            for file_path in files_to_delete:
                try:
                    os.unlink(file_path)
                    print(f"Deleted: {file_path}")
                except Exception as e:
                    print(f"Error deleting {file_path}: {e}")

if __name__ == "__main__":
    delete_files(root_dir, sim_folder_template, templates_to_delete)

In [20]:
delete_files(root_dir, sim_folder_template, templates_to_delete)